Grupo 12

Francisco de Borja Lozano del Moral

Manuel Ortega Salvador

## Tercera parte.  Recomendacion basada en filtrado colaborativo.

En esta tercera parte utilizaremos la librería SURPRISE Se puede consultar la documentacion en http://surpriselib.com/

Para instalarla: conda install -c conda-forge scikit-surprise o pip install numpy pip install scikit-surprise

La librería SurPRISE (Simple Python RecommendatIon System Engine) tiene algoritmos de predición de ratings incluidos: baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF) y otros. También tiene predefinidas las medidas de similitud mas comunes sobre vectores (cosine, MSD, pearson…) Una de las cosas más utiles es que proporciona herramientas para evaluar, analizar y comparar el rendimiento de distitnos algoritmos. Lo que vamos a hacer en esta parte de la práctica es probar varios procedimientos de evaluación cruzada midiendo datos sobre errores entre el valor real (conocido) y la predicción del recomendador. Las siglas corresponden a las siguientes medidas:

MAE: Mean Absolute Error
RMSE: Root mean square error (RMSE)
MSE: mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. -square root of the mean square error.

Vamos a ejecutar algunos recomendadores y evaluarlos para poder comentar los resultados.


In [1]:
from collections import defaultdict
import numpy as np

from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
## Ejemplo getting started de la documentación de SURPRISE
##http://surpriselib.com/

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9343  0.9332  0.9410  0.9352  0.9370  0.9361  0.0027  
MAE (testset)     0.7388  0.7359  0.7398  0.7367  0.7395  0.7381  0.0016  
Fit time          3.69    3.68    3.67    3.68    3.63    3.67    0.02    
Test time         0.14    0.10    0.13    0.13    0.13    0.13    0.01    


{'test_rmse': array([0.9343429 , 0.93316258, 0.94102922, 0.93515543, 0.93702904]),
 'test_mae': array([0.73877524, 0.73593063, 0.7398088 , 0.73666124, 0.73947574]),
 'fit_time': (3.692274570465088,
  3.679579019546509,
  3.674455404281616,
  3.6821165084838867,
  3.633521318435669),
 'test_time': (0.13609910011291504,
  0.10202169418334961,
  0.13121891021728516,
  0.130112886428833,
  0.12912511825561523)}

In [3]:
# Evaluacion extracted from surprise: 
# https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
def measures_at_k(predictions, k, th_recom, th_relev):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    onehits = dict()
    mrr = dict()
    
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= th_relev) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= th_recom) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= th_relev) and (est >= th_recom))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
       
        
    return precisions, recalls


In [4]:
def f1(precision, recall):
    """
        Funcion que calcula el f1 (media armónica) en funcion de precision y recall
    """
    denominador = precision + recall
    
    if denominador == 0:
        return 0
    else:
        return (2 * precision * recall) / denominador

In [5]:
def get_results(recommendations, k, knn):
    """
        Function to get the measures results 
    """
    # threshold = 4 --> solo se tienen en cuenta peliculas que hayan 
    # sido puntuadas con 4 o 5 estrellas
    precisions, recalls  = measures_at_k(recommendations, k, th_recom=4, th_relev=1)
    
    # Measures can then be averaged over all users
    precision_result = sum(prec for prec in precisions.values()) / len(precisions)
    recall_result = sum(rec for rec in recalls.values()) / len(recalls)
    # Media armónica  
    f1_result = f1(precision_result, recall_result)
    # En este archivo se volcarán los resultados de las métricas. Tiene que existir. 
    f = open("./results_user_cf.csv", 'a')
    #f = open("C:/hlocal/results_user_cf.csv", 'a')
    f.write(str(k) + ',' + knn + "," + str(precision_result) + ',' + str(recall_result) + ',' +  str(f1_result) +  '\n') 
    f.close()
    

In [6]:
# Hemos cargado antes los datos de movieLens para 100K
# data = Dataset.load_builtin('ml-100k')

In [7]:
# creo dos conjuntos de datos, el training set y el evaluation set
# cada uno contendra la mitad de los datos
training_set, evaluation_set = train_test_split(data, test_size=.5)

In [8]:
# Ahora determino cual es el algoritmo que voy a usar de recomendacion
# en este caso voy a usar el algoritmo KNN para encontrar las similitudes entre items
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})
#print(recommendation_algorithm)

In [9]:
# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)
#print(recommendation_algorithm)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [10]:
# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)
#print(recommendations)

In [11]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

In [12]:
##########################################################
# Hacer distintas pruebas con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [14]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## Ejercicio:  se pide ejecutar, comprender y escribir comentarios razonados sobre la evaluación de distintos recomendadores.
    
Prueba otros algoritmos de predicción de ratings basados en la estimación de los vecinos más próximos y realiza evaluaciones de su comportamiento. Comenta los resultados.¶
Puedes consultar la documentación en https://surprise.readthedocs.io/en/stable/knn_inspired.html#

### Resultados de lo anterior

De mejor a peor, parecemos tener:

1. baseline

2. wth zscore

3. wth means

4. basic

Esto, a primera vista tiene sentido. Cada usuario tiene un estilo de puntuación distinto, algunos siendo mucho más críticos que otros y solo dando notas entre 3-7, con otros dando 8-10 a todas las películas. Por ello, no tener en cuenta la media o distribución de notas va a funcionar peor.

Entre zscore y means, es predecible que funcione mejor asumir que los usuarios ponen notas que se asemejen a distribuciones normales, y que así tenga un peso correcto la distancia a la media. Que se tenga en cuenta de manera especial cuando un usuario le ha dado muchos puntos mayor a lo que suele poner es intuitivo.

Lo que es sorprendente es que una simple baseline funcione aun mejor. Asumimos que es porque al ser un parámetro aprendido, se ha encontrado una serie de baselines $b_u$ que funcionan aun mejor que asumiendo zscores a traves de experimentación, aprendiendo a disminuir la relevancia de algunos usuarios y aumentando la relevancia de otros, y de esta manera creando de alguna manera una calificación de como de fácil de satisfacer son algunos usuarios, y cómo de en serio hay que tomarlos.

Aunque el entrenamiento sería más lento, probablemente se lograría unos resultados aun mejores si se aprendiera la también un parámetro parecido a $\sigma_u$, para dar distinto valor según la distancia de $b_u$, pero este algoritmo no existe en Surprise.

|k |type                |P   |   R|  F1|
|--|--------------------|----|----|----|
|01|           knn_basic|0.95|0.04|0.07|
|02|           knn_basic|0.95|0.06|0.12|
|03|           knn_basic|0.95|0.09|0.16|
|04|           knn_basic|0.95|0.11|0.19|
|05|           knn_basic|0.95|0.12|0.22|
|06|           knn_basic|0.95|0.14|0.24|
|07|           knn_basic|0.95|0.15|0.26|
|08|           knn_basic|0.95|0.16|0.27|
|09|           knn_basic|0.95|0.17|0.28|
|10|           knn_basic|0.95|0.17|0.29|

|k |type                |P   |   R|  F1|
|--|--------------------|----|----|----|
|01|       knn_withmeans|0.86|0.03|0.06|
|02|       knn_withmeans|0.86|0.06|0.11|
|03|       knn_withmeans|0.86|0.08|0.15|
|04|       knn_withmeans|0.86|0.11|0.19|
|05|       knn_withmeans|0.86|0.13|0.22|
|06|       knn_withmeans|0.86|0.15|0.25|
|07|       knn_withmeans|0.86|0.16|0.27|
|08|       knn_withmeans|0.86|0.18|0.29|
|09|       knn_withmeans|0.86|0.19|0.31|
|10|       knn_withmeans|0.86|0.20|0.33|

|k |type                |P   |   R|  F1|
|--|--------------------|----|----|----|
|01|      knn_withzscore|0.85|0.03|0.06|
|02|      knn_withzscore|0.85|0.06|0.11|
|03|      knn_withzscore|0.85|0.08|0.15|
|04|      knn_withzscore|0.85|0.11|0.19|
|05|      knn_withzscore|0.85|0.13|0.22|
|06|      knn_withzscore|0.85|0.14|0.25|
|07|      knn_withzscore|0.85|0.16|0.27|
|08|      knn_withzscore|0.85|0.18|0.29|
|09|      knn_withzscore|0.85|0.19|0.31|
|10|      knn_withzscore|0.85|0.20|0.32|

|k |type                |P   |   R|  F1|
|--|--------------------|----|----|----|
|01|        knn_baseline|0.92|0.04|0.07|
|02|        knn_baseline|0.92|0.07|0.12|
|03|        knn_baseline|0.92|0.09|0.17|
|04|        knn_baseline|0.92|0.11|0.20|
|05|        knn_baseline|0.92|0.13|0.23|
|06|        knn_baseline|0.92|0.15|0.26|
|07|        knn_baseline|0.92|0.16|0.28|
|08|        knn_baseline|0.92|0.18|0.30|
|09|        knn_baseline|0.92|0.19|0.31|
|10|        knn_baseline|0.92|0.20|0.32|

### Pruebas con cosine
La comparación entre los algoritmos la haremos sobre los resultados obtenidos para el valor k=10, puesto que en todos ellos se aprecia claramente que desde k=1 hasta k=10 va aumentando el resultado de recall y por tanto la media, lo cual tiene sentido puesto que se están teniendo en cuenta un tamaño mayor de elementos en el set de recomendados y relevantes.

Primero, vamos a ver cómo afecta el computar las similitudes entre items en lugar de entre usuarios.

In [15]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [16]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': False})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


En KNNBasic y KNNWithMeans no ha habido mucha diferencia, mientras que en KNNWithZScore tenemos 0.3094456300468 de media con users y 0.32695845982079, y de nuevo en KNNBaseline la diferencia es muy pequeña. De todos modos en todas ellas, la similitud entre items nos ha dado una media mayor.

Ahora vamos a ver los mismos cuatro algoritmos empleando la similitud del coseno y viendo similitud entre users:

In [19]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'cosine', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [20]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'cosine', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [21]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'cosine', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [22]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'cosine', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


Por último. vamos a ver los mismos cuatro algoritmos empleando la similitud de la distancia euclídea y viendo similitud entre users:

In [23]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'msd', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

Computing the msd similarity matrix...
Done computing similarity matrix.


In [24]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'msd', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Computing the msd similarity matrix...
Done computing similarity matrix.


In [25]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'msd', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Computing the msd similarity matrix...
Done computing similarity matrix.


In [26]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'msd', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Puesto que los datos del fichero son muchos no comentaremos todos aquí uno por uno, y simplemente haremos comentarios generales de los resultados más remarcables que se han obtenido con estas pruebas.

Primero de todo, no ha habido diferencias remarcables al variar la similitud empleada para el mismo algoritmo KNN, por lo que dentro del mismo algoritmo no hay mucha diferencia entre aplicar uno que otro. Parece que los vecinos escogidos acaban siendo los mismos o muy parecidos independientemente de la similitud usada.

Por último, KNNWithMeans y KNNBaseline, de nuevo nos dan mejores resultados respecto a la media entre precision y recall. Explicable de nuevo con los razonamientos del apartado anterior.